<a href="https://colab.research.google.com/github/beyzaturku/image_processing_in_autonomous_vehicles/blob/main/serit_takip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q opencv-python

In [5]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from moviepy import editor
import moviepy
from google.colab import files
import os

In [7]:
print("Lütfen bir test videosu yükleyin:")
uploaded = files.upload()
test_video = list(uploaded.keys())[0]

Lütfen bir test videosu yükleyin:


Saving project_video.mp4 to project_video.mp4


In [8]:
import cv2
import numpy as np
import moviepy.editor as editor
import time

# 1. Renk eşikleme
def color_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)

    white = cv2.inRange(hls[:, :, 1], 200, 255)
    yellow = cv2.inRange(hls[:, :, 0], 15, 35) & cv2.inRange(hls[:, :, 2], 100, 255)
    light = cv2.inRange(lab[:, :, 0], 190, 255)
    grayish = cv2.inRange(hls, (0, 30, 60), (180, 80, 200))

    return cv2.bitwise_or(cv2.bitwise_or(cv2.bitwise_or(white, yellow), light), grayish)


# 2. Kenar tespiti
def edge_detection(gray):
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

# 3. ROI
def region_of_interest(img):
    h, w = img.shape[:2]
    mask = np.zeros_like(img)
    polygon = np.array([[
        (int(w * 0.1), h),
        (int(w * 0.4), int(h * 0.6)),
        (int(w * 0.6), int(h * 0.6)),
        (int(w * 0.9), h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

# 4. Hough
def hough_lines(img):
    lines = cv2.HoughLinesP(img, 1, np.pi/180, 50, minLineLength=40, maxLineGap=100)
    if lines is None:
        return []
    filtered = []
    for x1, y1, x2, y2 in lines.reshape(-1, 4):
        if x2 == x1: continue
        slope = (y2 - y1) / (x2 - x1)
        angle = abs(np.degrees(np.arctan(slope)))
        if 20 < angle < 80:
            filtered.append((x1, y1, x2, y2, slope))
    return filtered

# 5. Regresyon + merkez hesap + eğim yazdır
def fit_lane_lines(img, lines):
    h, w = img.shape[:2]
    left_points, right_points = [], []
    left_slope, right_slope = None, None

    for x1, y1, x2, y2, slope in lines:
        if slope < 0:
            left_points += [(x1, y1), (x2, y2)]
        else:
            right_points += [(x1, y1), (x2, y2)]

    def make_line(points):
        if not points:
            return None, None
        x, y = zip(*points)
        poly = np.polyfit(y, x, 1)

        y1 = int(h)
        y2 = int(h * 0.75)

        x1 = int(np.polyval(poly, y1))
        x2 = int(np.polyval(poly, y2))

        if not (0 <= x1 < w and 0 <= x2 < w):
            return None, None

        slope = -poly[0]
        return (x1, y1, x2, y2), slope

    left_line,  left_slope  = make_line(left_points)
    right_line, right_slope = make_line(right_points)

    lane_img = np.zeros_like(img)
    lane_center = None
    car_offset = None

    if left_line and right_line:
        cv2.line(lane_img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 255, 0), 8)
        cv2.line(lane_img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 0, 255), 8)

        left_base = left_line[0]
        right_base = right_line[0]
        lane_center = (left_base + right_base) // 2
        car_center = w // 2
        car_offset = (car_center - lane_center) * 3.7 / 700

        cv2.putText(lane_img, f"Slope L: {left_slope:.2f}", (left_line[0], left_line[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        cv2.putText(lane_img, f"Slope R: {right_slope:.2f}", (right_line[0], right_line[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    return lane_img, car_offset

last_time = [time.time()]
frame_counter = [0]

def frame_processor(image):
    start_time = time.time()
    debug_img = image.copy()

    frame_id = frame_counter[0]
    frame_counter[0] += 1
    os.makedirs("output", exist_ok=True)

    color_mask = color_threshold(image)
    cv2.imwrite(f"output/{frame_id:03d}_1_color_mask.jpg", color_mask)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edge_mask = edge_detection(gray)
    cv2.imwrite(f"output/{frame_id:03d}_2_edges.jpg", edge_mask)

    combined = cv2.bitwise_or(color_mask, edge_mask)
    cv2.imwrite(f"output/{frame_id:03d}_3_combined.jpg", combined)

    roi = region_of_interest(combined)
    cv2.imwrite(f"output/{frame_id:03d}_4_roi.jpg", roi)

    lines = hough_lines(roi)
    hough_img = np.zeros_like(image)
    for x1, y1, x2, y2, _ in lines:
        cv2.line(hough_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
    cv2.imwrite(f"output/{frame_id:03d}_5_hough_lines.jpg", hough_img)

    lane_img, offset = fit_lane_lines(image, lines)

    overlay = cv2.addWeighted(image, 0.8, lane_img, 1, 0)

    fps = 1.0 / (start_time - last_time[0])
    last_time[0] = start_time

    if offset is not None:
        direction = "left" if offset > 0 else "right"
        cv2.putText(overlay, f"Offset: {abs(offset):.2f}m {direction}", (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    cv2.imwrite(f"output/{frame_id:03d}_6_final_overlay.jpg", overlay)

    return overlay

def process_video(input_path, output_path):
    clip = editor.VideoFileClip(input_path)
    result = clip.fl_image(frame_processor)
    result.write_videofile(output_path, audio=False)

In [9]:
output_video = "sonuc_video.mp4"

# Çalıştırma
process_video(test_video, output_video)

files.download(output_video)

Moviepy - Building video sonuc_video.mp4.
Moviepy - Writing video sonuc_video.mp4



Moviepy - Done !
Moviepy - video ready sonuc_video.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>